In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import json

%matplotlib inline

In [3]:
def softmax(x, temp=1):
#     x should be as long as number of possible clicks, each possibility given a value (
#     the x.max() is there for stability
    ex = np.exp((x - x.max()) / temp)
    return ex / ex.sum()

In [4]:
cd ..\experiments\data\human\1.8

C:\Users\piichan\Documents\Berkeley\CoCoSci Lab\mcrl\MCRL\experiments\data\human\1.8


In [5]:
df=pd.read_csv('trials.csv', sep=',')

In [6]:
import ast
toList = lambda x: ast.literal_eval(x)
df['clicks'] = df['clicks'].apply(toList)

In [7]:
cd ..\..\..\exp1\static\json

C:\Users\piichan\Documents\Berkeley\CoCoSci Lab\mcrl\MCRL\experiments\exp1\static\json


In [8]:
with open('rewards_0.10.json') as f:
    rewards_low = json.load(f)

with open('rewards_1.25.json') as f:
    rewards_med = json.load(f)
    
with open('rewards_4.00.json') as f:
    rewards_high = json.load(f)

In [9]:
def get_rewards_for_trial(all_rewards, trial_id):
    for trial in all_rewards['train']:
        if trial['trial_id'] == trial_id:
            return trial['stateRewards']
    for trial in all_rewards['test']:
        if trial['trial_id'] == trial_id:
            return trial['stateRewards']

In [10]:
df.values[11]

array([11, 0, 1.25, 'none', 'none', 12, 6426.0, '[]', 15.0, 4,
       list([1, 5, 9, 13]), '[1272, 2030, 2884, 3586]', '[1, 13, 14, 15]',
       '[4572, 6033, 9770]'], dtype=object)

In [41]:
df

,Unnamed: 0,pid,info_cost,PR_type,message,trial_index,trial_id,delays,score,n_click,clicks,click_times,path,action_times
0,0,0,1.25,none,none,1,8969.0,[15],10.75,1,[1],[2869],"[1, 1, 2, 4]","[11219, 28273, 34591]"
1,1,0,1.25,none,none,2,9359.0,[15],0.25,3,"[5, 9, 1]","[1998, 4708, 11014]","[1, 9, 10, 12]","[13147, 32672, 35775]"
2,2,0,1.25,none,none,3,9876.0,[15],13.75,5,"[9, 12, 11, 15, 16]","[1647, 2669, 3494, 9504, 12246]","[1, 9, 10, 11]","[14037, 37723, 38677]"
3,3,0,1.25,none,none,4,6615.0,[15],-1.00,4,"[8, 16, 15, 13]","[1763, 6739, 7771, 8444]","[1, 13, 14, 15]","[10560, 26729, 28393]"
4,4,0,1.25,none,none,5,5157.0,[15],20.00,0,[],[],"[1, 5, 6, 8]","[3405, 23072, 66331]"
5,5,0,1.25,none,none,6,7410.0,[15],16.25,3,"[9, 1, 5]","[5647, 6733, 7580]","[1, 5, 6, 7]","[8692, 25672, 27321]"
6,6,0,1.25,none,none,7,6264.0,[15],8.25,3,"[1, 3, 4]","[1547, 2215, 2997]","[1, 13, 14, 15]","[5194, 21946, 22928]"
7,7,0,1.25,none,none,8,7618.0,[15],-6.50,2,"[9, 5]","[1690, 3103]","[1, 5, 6, 7]","[4244, 20830, 21732]"
8,8,0,1.25,none,none,9,8818.0,[15],10.75,1,[1],[1046],"[1, 1, 2, 3]","[2377, 18945, 19760]"
9,9,0,1.25,none,none,10,7003.0,[15],-15.75,3,"[1, 9, 5]","[1445, 2708, 3284]","[1, 1, 2, 4]","[4442, 21106, 21867]"


## Brute Force Approach

In [35]:
df.sort_values('info_cost')
low_cost = df.loc[df.info_cost == 0.1]
med_cost = df.loc[df.info_cost == 1.25]
high_cost = df.loc[df.info_cost == 4]

In [19]:
cost_type = low_cost
brfs_count = 0
dfs_count = 0
top = set([9, 10, 11, 12])
left = set([13, 14, 15, 16])
right = set([5,6,7,8])
bottom = set([1, 2, 3, 4])
for seq in cost_type['clicks']:
    if seq == []:
        continue     
    if len(seq) < 4:
        continue
    first_four = seq[:4]
    if set(first_four) == set([1,5,9,13]):
        if len(seq) > 4:
            rest = seq[4:]
            if len(rest) > 4:
                rest = seq[4:8]
            if set(rest) <= set([2,6,10,14]):
                brfs_count += 1 
        else:
            brfs_count += 1
    elif (set(first_four) == top)or(set(first_four) == right)or(set(first_four) == bottom)or(set(first_four) == left):
        if len(seq) > 4:
            rest = set(seq[4:])
            more = False
            if len(rest) > 4:
                more = True
                rest = set(seq[4:8])
            if (rest<=top)or(rest<=right)or(rest<=left)or(rest<=bottom):
                if more:
                    rest = set(seq[8:])
                    if len(rest) > 4:
                        rest = set(seq[8:12])
                    if (rest<=top)or(rest<=right)or(rest<=left)or(rest<=bottom):
                            dfs_count += 1                    
                else:
                    dfs_count += 1
        else:
            dfs_count += 1
    else: #between 1 and 3 clicks were made
        if (set(seq) < top)or(set(seq) < right)or(set(seq)< bottom) or (set(seq) < left):
            dfs_count += 1
        elif set(seq) < set([1, 5, 9, 13]):
            brfs_count += 1
        
(brfs_count/cost_type['clicks'].size) * 100

3.75

ver 0.991 

values are given in percentages; top to bottom is low, med, and high cost

i stands for initial (only the first 4 clicks) while f stands for full (entire sequence)

| Breadth-First | Depth-First(i)| Depth-First(f)|
| ------------- |:-------------:| -------------:|
| 3.02          | 12.50         |8.47       |
| 6.07          | 12.13         |    9.375      |
| 4.23          | 3.43          |    2.42       |



## Likelihood Model

In [11]:
def prune(threshold, state):
    #returns nodes on pruned branches
    pruned = []
    observed_states = np.nonzero(state)[0]
    bottom = [1, 2, 3, 4]
    top = [9, 10, 11, 12]
    right = [5, 6, 7, 8]
    left = [13, 14, 15, 16]
    direction = False
    if len(observed_states) != 0:
        for loc in observed_states:
            if (loc in [1, 2]) and (loc not in pruned):
                direction = bottom
            elif (loc in [5, 6]) and (loc not in pruned):
                direction = right
            elif (loc in [9, 10]) and (loc not in pruned):
                direction = top
            elif (loc in [13, 14]) and (loc not in pruned):
                direction = left
            if direction:
                path_val = np.sum(state[direction])
                if path_val < threshold:
                    pruned += direction
            else: #prune arm when both leaf nodes have a value below the threshold
                if (loc in [3, 7, 11, 15]) and (loc not in pruned):
                    if (state[loc] < threshold) and (state[loc+1] < threshold):
                        pruned += [loc-2, loc-1, loc, loc+1]
                elif (loc not in pruned):
                    if (state[loc] < threshold) and (state[loc-1] < threshold):
                        pruned += [loc-3, loc-2, loc-1, loc]
    return pruned        

In [12]:
#Note: Only continues depth-first based off of the last click 
#TODO: Check if half-explored arms are eventually explored to completion
def depth_first(state, cur_clicks, dist, strategy_t, prune_t):
    prob_next_click = np.zeros(17)
    observed_states = np.nonzero(state)[0] #works only because indices represent location
    possible_clicks = []
    if len(cur_clicks) == 0:
        possible_clicks += [1, 5, 9, 13]
    else:
        last_click = cur_clicks[-1]
        pruned_nodes = prune(prune_t, state)
        if last_click in [1, 5, 9, 13]:
            if last_click+1 in observed_states:
                for click in [last_click+2, last_click+3]:
                    if (click not in observed_states) and (click not in pruned_nodes):
                        possible_clicks.append(click)
                if len(possible_clicks) == 0:
                    for click in [1, 5, 9, 13]:
                        if (click not in observed_states) and (click not in pruned_nodes):
                            possible_clicks.append(click)
            else:
                if last_click+1 not in pruned_nodes:
                    possible_clicks.append(last_click+1)
                else:
                    click_candidates = [1, 5, 9, 13]
                    click_candidates.remove(last_click)
                    for c in click_candidates:
                        if (c in observed_states) or (c in pruned_nodes):
                            click_candidates.remove(c)
                    possible_clicks += click_candidates
        else:
            if last_click+1 in observed_states:
                if last_click+2 in observed_states:
                    for click in [1, 5, 9, 13]:
                        if (click not in observed_states) and (click not in pruned_nodes):
                            possible_clicks.append(click)
                elif (last_click+2 < 17) and (last_click+2 not in pruned_nodes):
                    possible_clicks.append(last_click+2)
                else:
                    for click in [1, 5, 9, 13]:
                        if (click not in observed_states) and (click not in pruned_nodes):
                            possible_clicks.append(click)
            elif (last_click+1 < 17) and (last_click+1 not in pruned_nodes):
                possible_clicks.append(last_click+1)
            else:
                for click in [1, 5, 9, 13]:
                        if (click not in observed_states) and (click not in pruned_nodes):
                            possible_clicks.append(click)
    if len(possible_clicks) != 0:
        for possibility in possible_clicks:
            if dist == 'Uniform':
                prob_next_click[possibility] = 1/len(possible_clicks)
            elif dist == 'Softmax':
                prob_next_click[possibility] = 1
    return prob_next_click

In [27]:
test_state = np.array([0,1,5,0,20,0,0,0,0,0,0,0,0,1,0,-10,-10])
res = depth_first(test_state, [], 'Softmax', None, -4)
res

array([ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.])

In [14]:
def breadth_first(state, cur_clicks, dist, strategy_t, prune_t):
    prob_next_click = np.zeros(17)
    observed_states = np.nonzero(state)[0]
    possible_clicks = []
    if len(cur_clicks) == 0:
        possible_clicks += [1, 5, 9, 13]
    else:
        last_click = cur_clicks[-1]
        pruned_nodes = prune(prune_t, state)
        if last_click in [1, 5, 9, 13]:
            if set([1, 5, 9, 13]).issubset(set(observed_states)):
                if set([2, 6, 10, 14]).issubset(set(observed_states)):
                    for click in [3, 4, 7, 8, 11, 12, 15, 16]:
                        if (click not in observed_states) and (click not in pruned_nodes):
                            possible_clicks.append(click)
                else:
                    for click in [2, 6, 10, 14]:
                        if (click not in observed_states) and (click not in pruned_nodes):
                                possible_clicks.append(click)
            else:
                for click in [1, 5, 9, 13]:
                    if (click not in observed_states) and (click not in pruned_nodes):
                            possible_clicks.append(click)
        elif last_click in [2, 6, 10, 14]:
            if set([2, 6, 10, 14]).issubset(set(observed_states)):
                for click in [3, 4, 7, 8, 11, 12, 15, 16]:
                    if (click not in observed_states) and (click not in pruned_nodes):
                        possible_clicks.append(click)
            else:
                for click in [2, 6, 10, 14]:
                    if (click not in observed_states) and (click not in pruned_nodes):
                        possible_clicks.append(click)
        else:
            for click in [3, 4, 7, 8, 11, 12, 15, 16]:
                    if (click not in observed_states) and (click not in pruned_nodes):
                        possible_clicks.append(click)
    if len(possible_clicks) != 0:
        for possibility in possible_clicks:
            if dist == 'Uniform':
                prob_next_click[possibility] = 1/len(possible_clicks)
            elif dist == 'Softmax':
                prob_next_click[possibility] = 1
    return prob_next_click

In [37]:
test_state = np.array([0,1,5,0,20,5,0,0,0,-3,0,0,0,1,0,-10,-10])
res = breadth_first(test_state, [], 'Uniform', None, 0)
res

array([ 0.  ,  0.25,  0.  ,  0.  ,  0.  ,  0.25,  0.  ,  0.  ,  0.  ,
        0.25,  0.  ,  0.  ,  0.  ,  0.25,  0.  ,  0.  ,  0.  ])

In [16]:
#assumes that average reward value is 0
def best_first(state, cur_clicks, dist, strategy_t, prune_t):
    prob_next_click = np.zeros(17)
    observed_states = np.nonzero(state)[0]
    possible_clicks = []
    pruned_nodes = prune(prune_t, state)
    expected_reward_value = 0 #or self.env.reward.expectation()
    if len(cur_clicks) == 0:
        possible_clicks += [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
    else:
        for i in np.arange(1,17):
            if (i not in observed_states) and (i not in pruned_nodes):
                possible_clicks.append(i)
    actual_state = np.copy(state)
    for i in np.arange(1, 17):
        if state[i] == 0:
            state[i] = expected_reward_value
    if len(possible_clicks) != 0:
        for possibility in possible_clicks:
            if possibility in [1, 5, 9, 13]:
                indices = [possibility+1, possibility+2]
                path_1 = np.sum(state[indices])
                indices[1] += 1
                path_2 = np.sum(state[indices])
                prob_next_click[possibility] = np.maximum(path_1, path_2)
            elif possibility in [2, 6, 10, 14]:
                indices = [possibility-1, possibility+1]
                path_1 = np.sum(state[indices])
                indices[1] += 1
                path_2 = np.sum(state[indices])
                prob_next_click[possibility] = np.maximum(path_1, path_2)
            elif possibility in [4, 8, 12, 16]:
                indices = [possibility-1, possibility-3]
                prob_next_click[possibility] = np.sum(state[indices])
            else:
                indices = [possibility-1, possibility-2]
                prob_next_click[possibility] = np.sum(state[indices])
    return prob_next_click

In [40]:
#state = np.array([0, 0, 20, 0, 0, 12, -8, 0, 0, -4, 0, 0, 0, 10, 0, 0, 0])
test_state = np.array([0,1,5,0,20,5,0,0,0,-3,0,0,0,1,0,-10,-10])
res = best_first(test_state, [1, 4, 16, 2, 13], 'Uniform', None, -30)
res

array([ 0.,  0.,  0.,  6.,  0.,  0.,  5.,  5.,  5.,  0., -3., -3., -3.,
        0., -9.,  0.,  0.])

In [18]:
def backwards_best(state, cur_clicks, dist, strategy_t, prune_t):
    prob_next_click = np.zeros(17)
    observed_states = np.nonzero(state)[0]
    possible_clicks = []
    pruned_nodes = prune(prune_t, state)
    leaves = [3, 4, 7, 8, 11, 12, 15, 16]
    goal = False
    goal_node = None
    if len(cur_clicks) == 0:
        possible_clicks += leaves
    else:
        for leaf in leaves: # should the leaves be examined in the order clicked?
            if (leaf-2 not in pruned_nodes) and (state[leaf] > strategy_t): #a goal node has been identified
                #check that the whole arm hasn't been explored before marking goal as true
                if leaf in [3, 7, 11, 15]:
                    if (leaf-1 not in observed_states) or (leaf-2 not in observed_states):
                        goal = True
                        goal_node = leaf
                        break
                else:
                    if (leaf-2 not in observed_states) or (leaf-3 not in observed_states):
                        goal = True
                        goal_node = leaf
                        break
        if goal:
            #go backwards on that arm
            if goal_node in [3, 7, 11, 15]:
                if goal_node-1 not in observed_states:
                    possible_clicks.append(goal_node-1)
                else:
                    possible_clicks.append(goal_node-2)
            else:
                if goal_node-2 not in observed_states:
                    possible_clicks.append(goal_node-2)
                else:
                    possible_clicks.append(goal_node-3)
        else:
            for leaf in leaves:
                if (leaf-2 not in pruned_nodes) and (leaf not in observed_states):
                    possible_clicks.append(leaf)
            
    if len(possible_clicks) != 0:
        for possibility in possible_clicks:
            if possibility in [1, 5, 9, 13]:
                indices = [possibility+1, possibility+2]
                path_1 = np.sum(state[indices])
                indices[1] += 1
                path_2 = np.sum(state[indices])
                prob_next_click[possibility] = np.maximum(path_1, path_2)
            elif possibility in [2, 6, 10, 14]:
                indices = [possibility-1, possibility+1]
                path_1 = np.sum(state[indices])
                indices[1] += 1
                path_2 = np.sum(state[indices])
                prob_next_click[possibility] = np.maximum(path_1, path_2)
            elif possibility in [4, 8, 12, 16]:
                indices = [possibility-1, possibility-3]
                prob_next_click[possibility] = np.sum(state[indices])
            else:
                indices = [possibility-1, possibility-2]
                prob_next_click[possibility] = np.sum(state[indices])
    return prob_next_click

In [33]:
test_state = np.array([0,1,5,0,20,5,0,0,0,-3,0,0,0,1,0,-10,-10])
res = backwards_best(test_state, [1, 4, 16, 2, 13], 'Softmax', 0, -30)
res

array([ 0.,  0.,  0.,  6.,  0.,  0.,  0.,  5.,  5.,  0.,  0., -3., -3.,
        0.,  0.,  0.,  0.])

In [19]:
def satisficing_stop(state, threshold):
    bottom = [1, 2, 3, 4]
    top = [9, 10, 11, 12]
    right = [5, 6, 7, 8]
    left = [13, 14, 15, 16]
    for direction in [bottom, top, right, left]:
        observed_value = np.sum(state[direction])
        if observed_value > threshold:
            return True
    return False    

In [46]:
def strategy_likelihood(click_seq, strategy, cost_per_click, rewards, p_error, soft_temp=1,
                        log=False, dist='Softmax', strategy_t=-20, prune_t=-30, stop=(0,0)):
    #Stop should be a tuple of (stopping rule type, threshold)
    if log:
        prob = 0
    else:
        prob = 1
    cur_state = np.zeros(17)
    cur_clicks = []
    stop_type = stop[0]
    stop_threshold = stop[1]
    for click in click_seq:
        possible_next_clicks = strategy(cur_state, cur_clicks, dist, strategy_t, prune_t)
        next_state = np.copy(cur_state)
        next_state[click] = rewards[click]
        cur_clicks.append(click)
        n_possible_clicks = np.count_nonzero(possible_next_clicks)
        if stop_type == 1: # number of clicks x cost per click
            if (len(cur_clicks)*cost_per_click) > stop_threshold:
                possible_next_clicks = np.zeros(17)
        elif stop_type == 2: #satisficing
            stop_flag = satisficing_stop(cur_state, stop_threshold)
            if stop_flag:
                possible_next_clicks = np.zeros(17)
        if dist == 'Uniform':
            p_strat = possible_next_clicks[click] 
            num_wrong_clicks = 16 - (np.count_nonzero(cur_state)+n_possible_clicks)
            p_click_given_state = (((1-p_error)*p_strat) + (p_error/num_wrong_clicks))
        elif dist == 'Softmax':
            prob_next_clicks = softmax(possible_next_clicks, soft_temp)
            p_click_given_state = prob_next_clicks[click]
        if log:
            prob += np.log(p_click_given_state)
        else:
            prob *= p_click_given_state
        cur_state = next_state
    return prob

In [47]:
participant = df.values[11]
rewards = get_rewards_for_trial(rewards_med, participant[6])
strategy_likelihood(participant[10], breadth_first, participant[2], rewards, 0.1, log=True)

-8.2176359933709033

In [52]:
all_likelihoods = np.zeros((len(df.values), 6))

p_error = 0.1
softmax_temp = 1
for i in np.arange(len(df.values)):
    trial = df.values[i]
    all_likelihoods[i, 0] = trial[1] #pid
    click_cost = trial[2]
    all_likelihoods[i, 1] = click_cost
    click_seq = trial[10]
    trial_id = trial[6]
    if click_cost == 0.10:
        rewards = get_rewards_for_trial(rewards_low, trial_id)
    elif click_cost == 1.25:
        rewards = get_rewards_for_trial(rewards_med, trial_id)
    elif click_cost == 4.00:
        rewards = get_rewards_for_trial(rewards_high, trial_id)
    strategies = [depth_first, breadth_first, best_first, backwards_best]
    for j in np.arange(4):
        strat = strategies[j]
        likelihood = strategy_likelihood(click_seq, strat, click_cost, rewards, p_error, softmax_temp)
        all_likelihoods[i, j+2] = likelihood

In [53]:
likelihood_df = pd.DataFrame(all_likelihoods, columns=['pid', 'click_cost', 'depth_first', 
                                                       'breadth_first', 'best_first', 'backwards_best' ])
likelihood_df

,pid,click_cost,depth_first,breadth_first,best_first,backwards_best
0,0.0,1.25,1.138637e-01,1.138637e-01,5.882353e-02,5.882353e-02
1,0.0,1.25,3.249774e-04,1.858222e-03,2.273118e-13,2.035416e-04
2,0.0,1.25,2.521249e-06,5.088294e-06,1.619881e-11,7.042963e-07
3,0.0,1.25,1.067335e-05,1.525640e-05,4.829714e-07,1.197304e-05
4,0.0,1.25,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
5,0.0,1.25,3.249774e-04,1.858222e-03,3.800713e-04,2.160157e-04
6,0.0,1.25,8.833803e-04,4.812760e-04,2.558689e-04,1.180224e-05
7,0.0,1.25,6.083019e-03,1.397047e-02,4.201681e-03,3.460208e-03
8,0.0,1.25,1.138637e-01,1.138637e-01,5.882353e-02,5.882353e-02
9,0.0,1.25,3.249774e-04,1.858222e-03,1.892802e-06,1.180224e-05


In [30]:
def strategy_features(state, last_click, action, expansion_rule):
    #returns an array of indicator values for each feature; value is 1 if action is compatible with the strategy
    ''' Features:
        [0] is_term
        [1] term_reward
        [2] voi_myopic
        [3] vpi_action
        [4] vpi_full
        [5] quality_ev
        [6] quality_std
        [7] depth
        [8] same_branch_as_last
    '''
    features = np.zeros(9)
    prob_next_click = expansion_rule(state, [last_click], 'Uniform', None, -30)
    func_name = expansion_rule.__name__
    if func_name == 'best_first':
        max_prob = np.maximum(prob_next_click)
        if prob_next_click[action] == max_prob:
            features[5] = 1
    elif func_name == 'breadth_first':
        if prob_next_click[action] > 0:
            features[7] = -1
    elif func_name == 'depth_first':
        if prob_next_click[action] > 0:
            features[6] = -1
            features[7] = -1
    
    return features

In [31]:
strategy_features(np.array([0, 1, 2, 0, 0, 0, 0,0,0,0,0,0,0,0,0,0,0]), 2, 3, depth_first)

array([ 0.,  0.,  0.,  0.,  0.,  0., -1., -1.,  0.])